In [ ]:
// range(0, 1000)
let numbers: number[] = Array.from(Array(1000).keys());

In [ ]:
function sumRecur(xs: number[]): number {
  if (xs.length < 1) {
    return 0
  }

  return xs[0] + sumRecur(xs.slice(1, xs.length))
}

In [ ]:
sumRecur(numbers)

# How does this evaluate?

```
let nums = [0, 1, 2, 3];
sumRecur(nums)
0 + sumRecur([1, 2, 3])
0 + (1 + sumRecur([2, 3]))
0 + (1 + (2 + sumRecur([3])))
0 + (1 + (2 + (3 + sumRecur([]))))
0 + (1 + (2 + (3 + 0)))
0 + (1 + (2 + 3))
0 + (1 + 5)
0 + 6
6
```

In [ ]:
function sumTailRecur(xs: number[]): number {
  function sumR(nums: number[], acc: number): number {
    if (nums.length < 1) {
      return acc
    }

    return sumR(nums.slice(1, nums.length), acc + nums[0])
  }

  return sumR(xs, 0)
}

In [ ]:
sumTailRecur(numbers)

# How does this evaluate?

```
let nums = [0, 1, 2, 3];
sumTailRecur(nums)
sumR([0, 1, 2, 3], 0)
sumR([1, 2, 3], 0)
sumR([2, 3], 1)
sumR([3], 3)
sumR([], 6)
6
```

# Tall call optimisation

In [ ]:
function sumNonRecur(xs: number[]): number {
  function sumNR(nums: number[], acc: number): number {
    while (true) {
      if (nums.length < 1) {
        return acc;
      }

      acc = acc + nums[0];
      nums = nums.slice(1, nums.length);
    }
  }

  return sumNR(xs, 0)
}

In [ ]:
sumNonRecur(numbers)

# Lots of useful functions can be built from recursion

In [ ]:
function lengthTailRecur(xs: number[]): number {
  function lengthR(nums: number[], acc: number): number {
    if (nums.length < 1) {
      return acc
    }

    return lengthR(nums.slice(1, nums.length), acc + 1)
  }

  return lengthR(xs, 0)
}

In [ ]:
lengthTailRecur(numbers)

In [ ]:
function maxTailRecur(xs: number[]): number {
  function maxR(nums: number[], acc: number): number {
    if (nums.length < 1) {
      return acc
    }

    return maxR(nums.slice(1, nums.length), acc > nums[0] ? acc : nums[0])
  }

  return maxR(xs, Number.MIN_VALUE)
}

In [ ]:
maxTailRecur(numbers)

In [ ]:
function minTailRecur(xs: number[]): number {
  function maxR(nums: number[], acc: number): number {
    if (nums.length < 1) {
      return acc
    }

    return maxR(nums.slice(1, nums.length), acc < nums[0] ? acc : nums[0])
  }

  return maxR(xs, Number.MAX_VALUE)
}

In [ ]:
minTailRecur(numbers)

# Getting DRY

In [ ]:
function recurser(xs: number[], f: (x: number, acc: number) => number, base: number): number {
  function helper(nums: number[], acc: number): number {
    if (nums.length < 1) {
      return acc
    }

    return helper(nums.slice(1, nums.length), f(acc, nums[0]))
  }

  return helper(xs, base)
}

# Folds beautiful folds (aka. reduce)

In [ ]:
function foldLeft(xs: number[], f: (acc: number, x: number) => number, base: number) {
  function helper(nums: number[], acc: number): number {
    if (nums.length < 1) {
      return acc
    }

    return helper(nums.slice(1, nums.length), f(acc, nums[0]))
  }

  return helper(xs, base)
}

In [ ]:
let add = (x: number, y: number): number => x + y
let mul = (x: number, y: number): number => x * y
let max = (x: number, y: number): number => x > y ? x : y
let min = (x: number, y: number): number => x < y ? x : y

console.log(foldLeft(numbers, add, 0))
console.log(foldLeft(numbers, mul, 1))
console.log(foldLeft([], max, Number.MIN_VALUE))
console.log(foldLeft(numbers, min, Number.MAX_VALUE))

# Corecursion

In [ ]:
function* range(start: number, end: number): Generator<number> {
  yield start;
  if (start + 1 >= end) {
    return;
  }

  yield* range(start + 1, end);
}

In [ ]:
Array.from(range(0, 10))

In [ ]:
import AWS from 'aws-sdk';

In [ ]:
const s3 = new AWS.S3();

In [ ]:
async function* listObjects(bucket: string, nct: string = undefined): AsyncGenerator<object, void, undefined> {
  const response = await s3.listObjectsV2({
    Bucket: bucket,
    ContinuationToken: nct,
    MaxKeys: 3,
  }).promise();

  const objs: Array<object> = response['Contents'];
  yield *objs;
  
  const nextToken = response['NextContinuationToken'];
  if (!nextToken) {
    return;
  }
  
  yield* listObjects(bucket, nextToken);
}

In [ ]:
async function testListObjects() {
  for await (const obj of listObjects('test-bucket')) {
    console.log(obj);
  }
}

In [ ]:
await testListObjects()

In [ ]:
function unfold<A, B>(f: (b: B) => Promise<[A[], B | null]>): (B) => AsyncGenerator<A, void, undefined> {
  return async function* unfolder(z: B) {
    let result = await f(z);
    yield* result[0];

    if (!result[1]) {
      return;
    }

    yield* unfolder(result[1]);
  }
}

In [ ]:
async function genRange({start, end}: {start: number, end: number}): Promise<[number[], object | null]> {
  if (start + 1 >= end) {
    return [[start], null ];
  } else {
    return [[start], {start: start + 1, end}];
  }
}

let range2 = unfold(genRange);

In [ ]:
async function listObjs({bucket, nct}: {bucket: string, nct: string}): Promise<[object[], object | null]> {
  const response = await s3.listObjectsV2({
    Bucket: bucket,
    ContinuationToken: nct || undefined,
    MaxKeys: 3,
  }).promise();

  const objs: Array<object> = response['Contents'];
  const nextToken = response['NextContinuationToken'];
  if (!nextToken) {
    return [objs, null];
  } else {
    return [objs, {bucket, nct: nextToken}];
  }
};

let listObjectsV2 = unfold(listObjs);

In [ ]:
async function testUnfoldRange() {
  for await (const i of range2({start: 0, end: 10})) {
    console.log(i);
  }
}

async function testUnfoldListObjects() {
  for await (const obj of listObjectsV2({bucket:"test-bucket"})) {
    console.log(obj);
  }
}

In [ ]:
await testUnfoldRange();

In [ ]:
await testUnfoldListObjects()